## 11.2 Retrofit - praca z URL

W poprzedniej aplikacji uzyskaliśmy dostęp do wszystkich postów, zazwyczaj potrzebujemy tylko jakiś podzbiór wszystkich dostępnych danych. W tym celu posługujemy się odpowiednimi parametrami w adresie URL. Ponownie wykorzystamy [**JSONPlaceholder**](https://jsonplaceholder.typicode.com/) - w sekcji **Rooutes** mamy podane różne rodzaje metod `HTTP`
- GET	/posts
- GET	/posts/1
- GET	/posts/1/comments
- GET	/comments?postId=1
- POST	/posts
- PUT	/posts/1
- PATCH	/posts/1
- DELETE	/posts/1

W tym przykładzie wyświetlimy wszystkie komentarze pod postem o zadanym identyfikatorze. Możemy to wykonać wykorzystując */posts/1/comments*
- posts - zwraca wszystkie posty
- 1 - identyfikator posta
- comments - zwraca wszystkie komentarze

Pamiętamy aby aplikacja miała dostęp do internetu oraz odpowiednie zależności.

Rozpoczniemy od modelu - format komentarzy wygląda następująco

In [ ]:
{
"postId": 1,
"id": 1,
"name": "id labore ex et quam laborum",
"email": "Eliseo@gardner.biz",
"body": "laudantium enim ..."
},

Więc tworzymy klasę o odpowiednich polach

In [ ]:
public class Comment {
    private int postId;
    private int id;
    private String name;
    private String email;

    @SerializedName("body")
    private String text;

    public int getPostId() {
        return postId;
    }

    public int getId() {
        return id;
    }

    public String getName() {
        return name;
    }

    public String getEmail() {
        return email;
    }
}

Utwórzmy nasz interfejs

In [ ]:
public interface PlaceholderService {

    @GET("posts/1/comments")
    Call<List<Comment>> getComments();
}

Utwórzmy `Retrofit` - tym razem jako singleton

In [ ]:
public final class RetrofitFactory {
    private RetrofitFactory(){}

    private static volatile PlaceholderService INSTANCE;
    private final String url = "https://jsonplaceholder.typicode.com/";

    public static PlaceholderService makeService() {
        if (INSTANCE == null) {
            synchronized (RetrofitFactory.class) {
                if (INSTANCE == null) {
                    INSTANCE = new Retrofit.Builder()
                            .baseUrl(url)
                            .addConverterFactory(GsonConverterFactory.create())
                            .build().create(PlaceholderService.class);
                }
            }
        }
        return INSTANCE;
    }
}

W aktywności głównej kod w niewielkim stopniu różnie się od poprzedniego przykładu

In [ ]:
        TextView textView = findViewById(R.id.textView);

        PlaceholderService service = RetrofitFactory.makeService();

        Call<List<Comment>> call = service.getComments();

        call.enqueue(new Callback<List<Comment>>() {
            @SuppressLint("SetTextI18n")
            @Override
            public void onResponse(
                @NonNull Call<List<Comment>> call, 
                @NonNull Response<List<Comment>> response) {
                if (response.isSuccessful()){
                    List<Comment> comments = response.body();
                    if (comments != null) {
                        comments.forEach(comment -> {
                            StringBuilder content = new StringBuilder();
                            content.append("id: ").append(comment.getId()).append("\n")
                                .append("PostId: ")
                                .append(comment.getPostId())
                                .append("\n")
                                .append("name: ")
                                .append(comment.getName())
                                .append("\n")
                                .append("email: ")
                                .append(comment.getEmail())
                                .append("\n")
                                .append("text: ")
                                .append(comment.getText())
                                .append("\n\n");
                            textView.append(content);
                        });
                    }
                } else
                    textView.setText("Code: " + response.code());
            }

            @Override
            public void onFailure(
                @NonNull Call<List<Comment>> call, 
                @NonNull Throwable t) {
                textView.setText(t.getMessage());
            }
        });

W odpowiedzi dostaniemy listę wszysktich komentarzy pod postem o identyfikatorze równym 1.

### **@Path**

Jeżeli chcemy napisać metodę przyjmującą jako parametr `id` posta, musimy wykorzystać odpowiednią adnotację - zmodyfikujmy metodę `getComments` w interfejsie `PlaceholderService`

In [ ]:
@GET("posts/{id}/comments")
Call<List<Comment>> getComments(@Path("id") int postId);

Dzięki zastosowaniu adnotacji `@Path` przekazujemy informację o zastosowaniu parametru metody jako składowej adresu `URL`. Tutaj musimy zwrócić uwagę na parametr `@Path` `"id"` - musi on być zgodny z częścią parametru adnotacji `@GET`, który podajemy w nawiasach `{id}`. Teraz wywołanie metody wygląda następująco

In [ ]:
Call<List<Comment>> call = service.getComments(3);

### **@Query**

Mamy również drugi sposób przekazania argumentu i uzyskania tej samej informacji - */comments?postId=1*. Dostaniemy dokładnie ta samą informację - wszystkie komentarza pod postem o zadanym `id`. Tym razem musimy wykorzystać adnotację `@Query` - zapytanie jest rozpoczynane symbolem `?`.

Dodajmy metodę `getCommentsFromQuery`

In [ ]:
@GET("comments")
Call<List<Comment>> getCommentsFromQuery(@Query("postId") int postId);

Tutaj parametr adnotacji `@Query` musi odpowiadać nazwie parametru (lub metody) obecnej w zapytaniu - tutaj będzie to */comments?postId=1*. Pozostałe elementy zapytania (znak rozpoczynający zapytanie oraz symbol `=`) zostanie dodany automatycznie. Metodę wywołujemy tak samo jak poprzednią

In [ ]:
Call<List<Comment>> call = service.getCommentsFromQuery(3);

Możemy również podać wiele parametrów - ich nazwy i wszystkie możliwości znadziemy w dokumentacji na stronie z  której korzystamy.

Chcemy uzyskać listę wszystkich komentarzy pod postem o zadanych `id`, posortowane po identyfikatorze malejąco. `url` będzie wyglądał następująco
- */comments?postId=1&_sort=id&_order=desc* - znak `&` rozdziela parametry

Napiszmy odpowiednią metodę

In [ ]:
@GET("comments")
Call<List<Comment>> getSortedComments(
        @Query("postId") int postId,
        @Query("_sort") String sort,
        @Query("_order") String order);

Wywołanie metody

In [ ]:
Call<List<Comment>> call = service.getSortedComments(2, "id", "desc");

Zwróćmy uwagę że parametr sortowania podajemy jakoi `String`.

Jeżeli chcemy dostać wszystkie komentarze posortowane malejąco po `id`, możemy zmodyfikować powyższą metodę i jako parametr `posrId` przekazać `null` - aby to zrobić musimy zamienić typ prymitywny `int` na obiekt `Integer`

In [ ]:
@GET("comments")
Call<List<Comment>> getNullableComments(
        @Query("postId") Integer postId,
        @Query("_sort") String sort,
        @Query("_order") String order);

Teraz chcąc wszystkie komentarze, możemy wywołać to funkcję

In [ ]:
Call<List<Comment>> call = service.getNullableComments(null, "id", "desc");

`null` można podać jako każdy argument - chcąc dostać wszystkie komentarze możemy wywołać metodę

In [ ]:
Call<List<Comment>> call = service.getNullableComments(null, null, null);

Jeżeli chcemy dostać listę komentarzy z kilku postów, możemy zadeklarować metodę przyjmującą tablicę (lub listę) identyfikatorów

In [ ]:
@GET("comments")
Call<List<Comment>> getSortedCommentsFromPosts(
        @Query("postId") Integer[] postsId,
        @Query("_sort") String sort,
        @Query("_order") String order);

Wtedy chcąc otrzymać posortowane komentarze z kilku postów, możemy metodę wywołać następująco

In [ ]:
Call<List<Comment>> call = service.getSortedCommentsFromPosts(
    new Integer[]{1, 3, 6, 8}, "id", "desc");

### **@QueryMap**

Jeżeli chcemy napisać metodę przyjmującą dowolną ilość parametrów, których nie określamy w definicji samej funkcji, możemy wykorzystać adnotację `@QueryMap`

In [ ]:
@GET("comments")
Call<List<Comment>> getComments(
        @QueryMap Map<String, String> param
);

Nasza metoda przyjmuje `Map` - pary klucz-wartość, gdzie kluczem typu `String` jest nazwa parametru. Wartości też podajemy jako `String`. Teraz w głównej aktywności możemy utworzyć mapę pożądanych parametrów i wykorzystać ją jako parametr funkcji.

In [ ]:
Map<String, String> param = new HashMap<>();
param.put("postId", "1");
param.put("_sort", "id");
param.put("_order", "desc");

Call<List<Comment>> call = service.getComments(param);

### **@Url**

Jeżeli adres jest skomplikowany z większą ilością parametrów, możemy chieć przekazać sam `url` jako parametr funkcji.

In [ ]:
@GET
Call<List<Comment>> getComments(
        @Url String url
);

Wtedy w wywołaniu metody `getComments` podajemy cały adres `url`

In [ ]:
Call<List<Comment>> call = service.getComments("comments?postId=3");

Możemy również podać

In [ ]:
Call<List<Comment>> call = service.getComments(
    "https://jsonplaceholder.typicode.com/comments?postId=3");

Na tym zakończymy omawianie podstaw pracy z adresami `url` oraz `GET`